In [1]:
import json
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import torch
from torch.utils.data import DataLoader

from candidate_extractors.TemplateMatchExtractor import TemplateMatchExtractor
from candidate_extractors.TemplateMatchExtractor import generate_dataset_from_slides
from datasets.SlideSeperatedCSVDataset import SlideSeperatedCSVDataset
from models.mlp import MLPBinaryClassifier
from models.resnet import Resnet18BinaryClassifier
from train import train_classifier
from utils import extract_features_from_dataset
from utils import plot_model_metrics
from utils import reduce_dataset, split_dataset
from datasets.SlideSeperatedImageDataset import SlideSeperatedImageDataset


In [2]:
slides_root_dir = "data/whole-slides/gut"
annotations_root_dir = "data/annotations/json"
candidates_dataset_dir = "output/candidates"
model_output_dir = "output/models"
PretrainedModelClass = Resnet18BinaryClassifier
features_csv_file_name = f"{PretrainedModelClass.get_pretrained_model_name()}_{PretrainedModelClass.pretrained_output_size}_features.csv"
print(f"{PretrainedModelClass.get_pretrained_model_name()}: {PretrainedModelClass.pretrained_output_size} features")

Resnet18: 512 features


In [3]:
extractor = TemplateMatchExtractor()
generate_dataset_from_slides(slides_root_dir, extractor, candidates_dataset_dir)

Found cached candidates dataset output/candidates


In [4]:
extract_features_from_dataset(candidates_dataset_dir,
                              [Resnet18BinaryClassifier])

Device: cuda:0
Found cached output/candidates/Resnet18_512_features.csv


In [5]:
def load_annotations(directory):
    slide_data = {"slide_name": [], "n_positive_annotations": []}
    for annotations_file in os.listdir(directory):
        with open(f"{directory}/{annotations_file}") as f:
            annotations = json.load(f)
            slide_name = Path(annotations_file).stem
            slide_data["slide_name"].append(slide_name)
            slide_data["n_positive_annotations"].append(len(annotations))
    return pd.DataFrame(slide_data)


def assign_categories(dataframe):
    q1, median, q3 = dataframe['n_positive_annotations'].quantile([0.25, 0.5, 0.75])

    def categorize_quartiles(n_annotations):
        if n_annotations <= q1:
            return "Low"
        elif q1 < n_annotations <= median:
            return "Medium"
        elif median < n_annotations <= q3:
            return "High"
        else:
            return "Very High"

    dataframe['category'] = dataframe['n_positive_annotations'].apply(lambda x: categorize_quartiles(x))
    return dataframe


def split_data(dataframe, train_portion=0.7):
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()
    for category in dataframe['category'].unique():
        category_slides = dataframe[dataframe['category'] == category]
        train_samples = category_slides.sample(frac=train_portion)
        test_samples = category_slides.drop(train_samples.index)
        train_set = pd.concat([train_set, train_samples])
        test_set = pd.concat([test_set, test_samples])
    return train_set, test_set


slides_df = load_annotations(annotations_root_dir)
slides_df = assign_categories(slides_df)
train_slides, test_slides = split_data(slides_df)
print("Train Slides")
train_slides

Train Slides


,slide_name,n_positive_annotations,category
15,593448,13,Low
10,593441,4,Low
0,522021,1,Low
2,593433,3,Low
1,522934,129,Very High
12,593445,100,Very High
16,593449,226,Very High
4,593435,91,Very High
20,593453,18,Medium
14,593447,25,Medium


In [6]:
print("Test Slides")
test_slides

Test Slides


,slide_name,n_positive_annotations,category
13,593446,15,Low
18,593451,1,Low
5,593436,151,Very High
19,593452,167,Very High
8,593439,27,Medium
11,593444,30,High


In [7]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

batch_size = 4096
# dataset = SlideSeperatedImageDataset(candidates_dataset_dir, set(train_slides["slide_name"]))
dataset = SlideSeperatedCSVDataset(f"{candidates_dataset_dir}/{features_csv_file_name}",
                                   set(train_slides["slide_name"]))
dataset = reduce_dataset(dataset, discard_ratio=0.0)
train_dataset, validation_dataset = split_dataset(dataset, train_ratio=0.9)
# train_dataset = undersample_dataset(train_dataset)

train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
validation_loader = DataLoader(validation_dataset,
                               batch_size=batch_size,
                               shuffle=True, )

# model = Resnet18BinaryClassifier(hidden_layers=1, units_per_layer=2048,
#                       dropout=0.3, focal_alpha=0.9, focal_gamma=2.0)
model = MLPBinaryClassifier(in_features=PretrainedModelClass.pretrained_output_size, hidden_layers=2,
                            units_per_layer=PretrainedModelClass.pretrained_output_size,
                            dropout=0.3, focal_alpha=0.85, focal_gamma=4.0)

print(f"Dataset: {len(train_dataset):,} training, {len(validation_dataset):,} validation")


Device: cuda:0
Dataset: 111,977 training, 12,442 validation


In [8]:
print(model)

MLPBinaryClassifier(
  (model): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=512, out_features=1, bias=True)
    (10): Sigmoid()
  )
)


In [9]:
print(f"Training starts {datetime.now().isoformat()}")

Training starts 2025-02-04T23:58:04.674577


In [10]:

model = model.to(device)
model, model_metrics = train_classifier(model, train_loader, validation_loader, device,
                                        start_learning_rate=0.000050,
                                        min_learning_rate=0.000025,
                                        lr_warmup_steps=20,
                                        max_epochs=30,
                                        checkpoint_every=1,
                                        eval_every=1)


Epoch 1 training: 100%|██████████| 28/28 [00:01<00:00, 16.83it/s]


Train: 1/30: lr: 0.000050000000 loss:0.002735402127395251


Epoch 1 testing: 100%|██████████| 4/4 [00:00<00:00, 15.67it/s]


Test: 1/30: loss:0.001641034905333072, accuracy:0.99298095703125, precision:0.0, recall:0.0, f1:0.0, mcc:0.0, ece:0.00701904296875, epoch:1


Epoch 2 training: 100%|██████████| 28/28 [00:01<00:00, 19.72it/s]


...

Test: 29/30: loss:0.0009605345403542742, accuracy:0.9650767933238636, precision:0.13014756216686024, recall:0.46230811403508765, f1:0.20184590690208667, mcc:0.2360062415516005, ece:0.03492320701479912, epoch:29


Epoch 30 training: 100%|██████████| 28/28 [00:02<00:00, 10.74it/s]


Train: 30/30: lr: 0.000025000000 loss:0.0009437822071569306


Epoch 30 testing: 100%|██████████| 4/4 [00:00<00:00, 11.80it/s]

Test: 30/30: loss:0.001146105263615027, accuracy:0.9700571035409903, precision:0.14244908097367115, recall:0.4384920634920635, f1:0.21488076743824763, mcc:0.23851560320880855, ece:0.029942896217107773, epoch:30


In [ ]:


print(f"Training ends {datetime.now().isoformat()}")
for metric in ["accuracy", "precision", "recall", "f1", "mcc"]:
    if len(model_metrics[f"test_{metric}"]) > 0:
        print(f"Test {metric}:", model_metrics[f"test_{metric}"][-1])

plot_model_metrics(model_metrics)

Training ends 2025-02-04T23:59:20.901146
Test accuracy: 0.9700571035409903
Test precision: 0.14244908097367115
Test recall: 0.4384920634920635
Test f1: 0.21488076743824763
Test mcc: 0.23851560320880855


In [24]:
os.makedirs(model_output_dir, exist_ok=True)
torch.save(model, f"{model_output_dir}/model.pickle")
torch.save({
    "train_slides": set(train_slides["slide_name"]),
    "test_slides": set(test_slides["slide_name"])
},
    f"{model_output_dir}/data-split.pickle"
)